In [1]:
from keras.applications import ResNet50V2
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Training Custom Connected Layer w/ResNet50V2 Base

In [2]:
datagen = ImageDataGenerator(rescale=1./255, 
                             brightness_range=[0.8,1.2],
                             rotation_range=30,
                             horizontal_flip=True,
                             zoom_range=[0.8,1.2],
                             fill_mode='nearest',
                             validation_split=0.03)

train_datagen = datagen.flow_from_directory('data/train/', seed=42, class_mode='categorical', subset='training', target_size=(512,512))
# prints "XXX images belonging to 101 classes"
val_datagen = datagen.flow_from_directory('data/train/', seed=42, class_mode='categorical', subset='validation', target_size=(512,512)) 
# prints "XXX images belonging to 101 classes"

train_steps = len(train_datagen) #1894
val_steps = len(val_datagen) #474
classes = len(list(train_datagen.class_indices.keys())) #101

Found 73528 images belonging to 102 classes.
Found 2222 images belonging to 102 classes.


In [3]:
conv_base = ResNet50V2(weights='imagenet', include_top=False, pooling='avg', input_shape=(512, 512, 3))

In [4]:
model = Sequential()

model.add(conv_base)
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(classes, activation='softmax'))

conv_base.trainable = False

In [5]:
import math

def lr_schedule(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [6]:
callbacks = [
    ModelCheckpoint(
        filepath='trained_top_resnet.h5',
        monitor='val_acc',
        save_best_only=True,
    ),
    EarlyStopping(
        monitor='val_acc',
        patience=2,
    )
]

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['acc','top_k_categorical_accuracy'])

In [8]:
history = model.fit_generator(
    train_datagen,
    steps_per_epoch=train_steps,
    epochs=50,
    verbose=2,
    validation_data=val_datagen,
    validation_steps=val_steps,
    callbacks=callbacks
)

Epoch 1/50
 - 5927s - loss: 2.8424 - acc: 0.3389 - top_k_categorical_accuracy: 0.6004 - val_loss: 2.4957 - val_acc: 0.4028 - val_top_k_categorical_accuracy: 0.6962
Epoch 2/50
 - 5420s - loss: 2.3840 - acc: 0.4247 - top_k_categorical_accuracy: 0.6945 - val_loss: 2.7581 - val_acc: 0.4203 - val_top_k_categorical_accuracy: 0.6962
Epoch 3/50
 - 5647s - loss: 2.3327 - acc: 0.4348 - top_k_categorical_accuracy: 0.7059 - val_loss: 3.3645 - val_acc: 0.4334 - val_top_k_categorical_accuracy: 0.7030
Epoch 4/50
 - 5855s - loss: 2.3134 - acc: 0.4382 - top_k_categorical_accuracy: 0.7082 - val_loss: 2.1044 - val_acc: 0.4176 - val_top_k_categorical_accuracy: 0.6962
Epoch 5/50
 - 5889s - loss: 2.3103 - acc: 0.4396 - top_k_categorical_accuracy: 0.7077 - val_loss: 2.0695 - val_acc: 0.4347 - val_top_k_categorical_accuracy: 0.7025
Epoch 6/50
 - 5761s - loss: 2.3109 - acc: 0.4399 - top_k_categorical_accuracy: 0.7086 - val_loss: 2.8691 - val_acc: 0.4401 - val_top_k_categorical_accuracy: 0.7084
Epoch 7/50
 - 57

# Visualize Performance Over Time

In [1]:
import matplotlib.pyplot as plt

def plot_history(history, metric):
    plt.title(metric)
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.ylabel(metric)
    plt.xlabel('epoch #')
    plt.legend(['train '+metric,'validation '+metric], loc='best')
    plt.show()

In [ ]:
plot_history(history, 'acc')

In [ ]:
plot_history(history, 'loss')